In [98]:
import os 
import openai
from time import sleep
from tqdm import tgrange, tqdm_notebook

#Bloc
MESSAGES = [{"role":"system", "content": "You're a Chatbot, your name is AlexBot you've been created by Alexandre Ohayon, you can answer any customer's questions by summarizing your answer."}]
#Dialog Box ask your question to the ChatBot:

try:
    print("AlexBot is ready. Tape 'exit' to quit the conversation")

    #Start a session:
    while True:
        #Enter your question:
        USER_PROMPT = input("Enter your message here: ") #Use the dialog box
        #Tape exist in lower character to quit the conversation with the Bot.
        if USER_PROMPT.lower() == "exit": 
            print("Thanks for choosing AlexBot. See you soon !") #Message from system
            break #Session closed
            
        #If a conversation exists then add user question to the Bloc in line 7:
        MESSAGES.append({"role":"user", "content": USER_PROMPT}) #Add the question into the list:
        user_question = {"role":"user", "content": USER_PROMPT}
        #print(f"User_question : {USER_PROMPT}")

                
        #Pass MESSAGES through the LLM to get response from the AI Agent:
        client = openai.OpenAI(organization=None,
                                project=None, 
                                timeout=60*5, #5 minutes and break after that
                                max_retries=2,
                                api_key=API_KEY)
            
        #Response from the Intelligent Agent:
        # Add the appropriate parameters to the decorator
        response = client.chat.completions.create(
            model='gpt-4o-mini',
            messages=MESSAGES,
            max_tokens=300,
            temperature=0,
            top_p=0.9,
            stop=["\n"]
        )

        assistant_dict = {"role":"assistant", "content": response.choices[0].message.content}
        print(f"ChniniBot : {assistant_dict["content"]}")
        print("--------------------------------------------------")
        #Add the question into the list:
        MESSAGES.append(assistant_dict)

except Exception as e:
    print("Error :", e)

AlexBot is ready. Tape 'exit' to quit the conversation
ChniniBot : Salut ! Comment puis-je vous aider aujourd'hui ?
--------------------------------------------------
Thanks for choosing AlexBot. See you soon !


In [101]:
import openai
import os

# Initialisation du client OpenAI avec la clé API (via variable d'environnement)
client = openai.OpenAI(organization=None,
                                project=None, 
                                timeout=60*5, #5 minutes and break after that
                                max_retries=2,
                                api_key=API_KEY) # Remplace si besoin

def get_famous_dishes_by_country():
    # Prompt demandé
    prompt = (
        "Donne-moi une liste des 10 plats les plus connus dans le monde, "
        "triée par pays, au format JSON. Pour chaque plat, indique le nom du plat et le pays d'origine. "
        "Utilise ce format : "
        '{ "plats": [ { "nom": "Sushi", "pays": "Japon" }, ... ] }'
    )

    # Appel à l'API avec la méthode chat.completions.create
    response = client.chat.completions.create(
        model="gpt-4o",  # ou "gpt-4"
        messages=[
            {"role": "system", "content": "Tu es un assistant culinaire qui fournit des réponses en JSON."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.3,
        max_tokens=500,
        response_format={"type":"json_object"} # Nécessite GPT-4-turbo ou GPT-4o
    )

    return response.choices[0].message.content

# Utilisation
if __name__ == "__main__":
    json_result = get_famous_dishes_by_country()
    print(json_result)


{
  "plats": [
    { "nom": "Sushi", "pays": "Japon" },
    { "nom": "Pizza", "pays": "Italie" },
    { "nom": "Tacos", "pays": "Mexique" },
    { "nom": "Couscous", "pays": "Maroc" },
    { "nom": "Paella", "pays": "Espagne" },
    { "nom": "Croissant", "pays": "France" },
    { "nom": "Hamburger", "pays": "États-Unis" },
    { "nom": "Poutine", "pays": "Canada" },
    { "nom": "Kimchi", "pays": "Corée du Sud" },
    { "nom": "Pad Thai", "pays": "Thaïlande" }
  ]
}
